<a href="https://colab.research.google.com/github/duyvm/leetcode-problems/blob/main/%5BHAR%5D3343_Count_Number_of_Balanced_Permutations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3343. Count Number of Balanced Permutations

https://leetcode.com/problems/count-number-of-balanced-permutations/description/

You are given a string `num`. A string of digits is called **balanced** if the **sum** of the digits at **even** indices is equal to the **sum** of the digits at **odd** indices.

Return the **number of distinct permutations** of `num` that are balanced.

Since the answer may be very large, return it `modulo 10`<sup>`9`</sup>` + 7`.

A **permutation** is a rearrangement of all the characters of a string.

**Constraints:**
- `2 <= num.length <= 80`
- `num` consists of digits `'0'` to `'9'` only.

**Example 1**

*Explanation*: The distinct permutations of num are "123", "132", "213", "231", "312" and "321".
Among them, "132" and "231" are balanced. Thus, the answer is 2.

In [1]:
test_case1 =  {
            "input": {
                "num": "123"
            },
            "output": 2
        }

**Example 2**

*Explanation*: The distinct permutations of num are "112", "121", and "211".
Only "121" is balanced. Thus, the answer is 1.

In [2]:
test_case2 =  {
            "input": {
                "num": "112"
            },
            "output": 1
        }

**Example 3**

*Explanation*: None of the permutations of num are balanced, so the answer is 0.

In [3]:
test_case3 =  {
            "input": {
                "num": "12345"
            },
            "output": 0
        }

In [4]:
test_cases = [test_case1, test_case2, test_case3]

In [5]:
def run_test_cases(solution, function_name):
    for i in range(len(test_cases)):
        run_test_case(solution, function_name, i)

def run_test_case(solution, function_name, i):
    test_case = test_cases[i]
    result = getattr(solution, function_name)(**test_case["input"])
    if result != test_case["output"]:
        print(f'Failed test case {i} with input: {test_case["input"]} and expected result: {test_case["output"]} vs actual result: {result}')

# Approach 1 - Dynamic Programming + Combinatorial (Beat 21%)

## Observations

####At first glance

- A string digit `num` with length `n`

- With `num`, we have total number of permutations: `total_permutations = n!`

  - Including duplicated digit.
  
  - For example `num = 131`, we have total `6` permutations with dupcated `131`, `113`, `311`, `311`, `131`, `113`

- The problem is to find total number of **distinct** `balanced_permutation`, a subset of `total_permutations`.

- Permutation is called balanced if sum of digits in odd indices `oddSum` is equal to sum of digits in even indices `evenSum`

- Furthermore, the answer requires the number of **distinct** `balanced_permutation`, which means we must remove duplicated string number if exist

####More

- Let `sumDigits` is sum of all digits of `num`. To make `oddSum` and `evenSum` equal, `sumDigits` must be even. We can check if `sumDigits` is odd, we can return `0` immediately. It mean there is no balanced permutation of `num`

- If `sumDigits` is even. We have

    $$haftSum = sumDigits ÷ 2$$

- Let `k` is the number of digits in odd indices (`1,3,5...`). Because the first digit start from even index (`0`), `k = n // 2`, regardless of `n` is even or odd.

- The original problem turn into find number of combinations of selected `k` digits from a set of `n` digits  $\binom{n}{k}$ that sum of chosen group digits `sumChosenDigits` is equal to `haftSum`. The un-chosen digits will be placed at even indices and have their sum is also `haftSum`. It is also the total number of balanced permutations of `num`

  - Because the problem requires return the number of **distinct** permutations, we must remove duplicated permutation from above result.

####Using tabulation dynamic programming to find number of ways to choose k digits that sum of their values equals to haft of total sum

- As stated above, given `sumDigits` is even, we need to find `k` digits, which are at odd indices, from the set of `n` digits, so that sum of `k` digits is equal to `haftSum`.

- We treat each digit `d` in `nums` as distinct, even they have same value. So their combinations will be different. For example, let `num=1122`, to choose `2` digits that sum of their values is equal to `3`, the combinations of `1st` and `3rd` digit (`"12"`) and `1st` and `4th` digit (also `"12"`) are different conbinations.

- We can achieve this by using dynamic programming with tabulation approach.

- Let `dp` with size `(haftSum + 1) x (k+1)` is **2D-array** that store information of states. At `dp[i][j]`, we have state information as follow:

  - `i`: sum of current chosen digits

  - `j`: number of current chosen digits

  - `d[i][j]`: number of ways that we can choose `j` digits from `n` distinct digits that sum of all `j` digits is `i`

  - The final state is we result need to find: `d[haftSum][k]`, which is number of ways that we can choose `k` digits so that sum of their values is equal to `haftSum`

- We fill up `dp` as follow
  
  - For `i` from `haftSum` to `d` and `j` from `k` to `1`: if we include `d` in current state `dp[i][j]`, the number of ways to achieve `i` by `j` digits is calculated by `dp[i][j] = dp[i][j] + d[i-d][j-1]`.

  - In other word, the new number of ways to achieve sum `i` by `j` digits (updated `dp[i][j]`) is equal to number of ways to achieve sum `i` by `j` digits without `d` (previous `dp[i][j]`) plus number of ways to achieve sum `i-d` with `j-1` (`dp[i-d][j-1]`). Note that because we include `d`, so we compute from `dp[i-d][j-1]`

  - The reason we start from `haftSum` and `k` is to make sure that we only consider `d` once for each `dp[i][j]` by adding it with previous state of `d[i][j]` as we moving from `dp[halfSum][k]` toward `d[0][0]`.

  - After consider all digit in `nums`, the number of ways to find `k` digits from `n` that sum to `haftSum` is `dp[haftSum][k]`.
  
  - Furthermore, after we found out all the number of ways to find `k` digits from `n` digits that sum of their values is `haftSum`, which is `dp[haftSum][k]`. Then:

    - For each way, we pick `k` digits for odd indices. It would be `k!` permutations for odd indices. And we have `n-k` digits for even indices, it would be `(n-k)!` permutations for even indices.
   
    - So the total permutations for `dp[haftSum][k]` ways is:
    
    $$dp[haftSum][k]×k!×(n-k)!$$

####Concrete Example

- Given `nums` = `1144`, `k` = `2`, `haftSum` = `5`, `dp` has size of `[6 x 3]`

- Initialized state of `dp`, only `dp[0][0] = 1`

>>>>  | | | |
|-|-|-|
| 1 | 0 | 0 |
| 0 | 0 | 0 |
| 0 | 0 | 0 |
| 0 | 0 | 0 |
| 0 | 0 | 0 |
| 0 | 0 | 0 |

  - First digit `d = num[0] = 1`:
    
    - For `i` from `5` to `d` and `j` from `k` to `1`, `dp[i][j] = dp[i][j] + dp[i-1][j-1]`

    - With only 1st digit for consideration, there ony one way to choose one digit that sum up to `1` (`dp[1][1]=1`)

>>>>  | | | |
|-|-|-|
| 1 | 0 | 0 |
| 0 | 1 | 0 |
| 0 | 0 | 0 |
| 0 | 0 | 0 |
| 0 | 0 | 0 |
| 0 | 0 | 0 |

  - Second digit `d = num[1] = 1`

    - For `i` from `5` to `d` and `j` from `k` to `1`, `dp[i][j] = dp[i][j] + dp[i-1][j-1]`

    - With  1st and 2nd digits for consideration, there are two ways to choose one digit that sum up to `1` (`dp[1][1]=2`, because each digit in `num` is distinct, one way for each digit `"1"`) and one way to choose two digits that sum up to `2` (`dp[2][2]=1`, it is `"11"`)

>>>>  | | | |
|-|-|-|
| 1 | 0 | 0 |
| 0 | 2 | 0 |
| 0 | 0 | 1 |
| 0 | 0 | 0 |
| 0 | 0 | 0 |
| 0 | 0 | 0 |

  - Third digit `d = num[2] = 4`

    - For `i` from `5` to `d` and `j` from `k` to `1`, `dp[i][j] = dp[i][j] + dp[i-4][j-1]`

    - With adding 3rd digit for consideration, we update two states in the `dp` table: `dp[5][2]=2` (two ways to choose two digits from considering digits that sum of their values is equal to `5`: `"41"` and `"41"` - remember that 1st and 2nd digits are distinct eventhough they have same value), `dp[4][1]=1` (one way to choose one digits from considering digits its value is equal to `4`: `"4"`)

>>>>  | | | |
|-|-|-|
| 1 | 0 | 0 |
| 0 | 2 | 0 |
| 0 | 0 | 1 |
| 0 | 0 | 0 |
| 0 | 1 | 0 |
| 0 | 0 | 2 |

  - Fourth digit `d = num[3] = 4`

   - For `i` from `5` to `d` and `j` from `k` to `1`, `dp[i][j] = dp[i][j] + dp[i-4][j-1]`

   - With adding the last digit for consideration, we have the final result for choosing `k=2` digits from `n=4` digits that sum of their values is equal to `haftSum=5`: `dp[5][4]=4` (two distinct digits `"4"` and two distinct digits `"1"`)


>>>>  | | | |
|-|-|-|
| 1 | 0 | 0 |
| 0 | 2 | 0 |
| 0 | 0 | 1 |
| 0 | 0 | 0 |
| 0 | 2 | 0 |
| 0 | 0 | 4 |

  - Finally, we have total permutations:

    $$dp[5][2]×2!×(4-2)! = 4×2×2=16$$

####How can we remove duplicated string?

- Since, the problem requires return the number of **distinct** permutation. We must remove duplicated string from the result above.

- Let `frequency` is the number of digit `d` appear in `num`. For each duplicating digit `d`, we must divide the above result for `frequency!` to remove duplicating string.

- For above example, the result before removal of duplicating string is `16`. Then we divide by two of `2!` (once for `1` and once for `2`, because each digit has two occurences in `num`). The final result:

$$16÷2!÷2! = 16÷2÷2=4$$

- Visualization: We have `num` = <font color='red'>1</font><font color='blue'>1</font><font color='red'>4</font><font color='blue'>4</font> (different color indicate different digit). As computed by `dp` above, we have `4` ways to pick `2` digits, each way have `4` permutations, so we have total `16` balanced permutations:
      
| | |  |  |  |  |
| - | - |-|-| - | - |
| odd indices: <font color='red'>1</font><font color='red'>4</font> and even indices: <font color='blue'>1</font><font color='blue'>4</font> | | <font color='blue'>4</font><font color='red'>1</font><font color='blue'>1</font><font color='red'>4</font> | <font color='blue'>1</font><font color='red'>4</font><font color='blue'>4</font><font color='red'>1</font> | <font color='blue'>4</font><font color='red'>4</font><font color='blue'>1</font><font color='red'>1</font> | <font color='blue'>1</font><font color='red'>1</font><font color='blue'>4</font><font color='red'>4</font> |
| odd indices: <font color='red'>1</font><font color='blue'>4</font> and even indices: <font color='blue'>1</font><font color='red'>4</font> |  | <font color='red'>4</font><font color='red'>1</font><font color='blue'>1</font><font color='blue'>4</font> | <font color='blue'>1</font><font color='blue'>4</font><font color='red'>4</font><font color='red'>1</font> | <font color='red'>4</font><font color='blue'>4</font><font color='blue'>1</font><font color='red'>1</font> | <font color='blue'>1</font><font color='red'>1</font><font color='red'>4</font><font color='blue'>4</font> |
| odd indices: <font color='blue'>1</font><font color='red'>4</font> and even indices: <font color='red'>1</font><font color='blue'>4</font> |  | <font color='blue'>4</font><font color='blue'>1</font><font color='red'>1</font><font color='red'>4</font> | <font color='red'>1</font><font color='red'>4</font><font color='blue'>4</font><font color='blue'>1</font> | <font color='blue'>4</font><font color='red'>4</font><font color='red'>1</font><font color='blue'>1</font> | <font color='red'>1</font><font color='blue'>1</font><font color='blue'>4</font><font color='red'>4</font> |
| odd indices: <font color='blue'>1</font><font color='blue'>4</font> and even indices: <font color='red'>1</font><font color='red'>4</font> |  | <font color='blue'>4</font><font color='red'>1</font><font color='blue'>1</font><font color='red'>4</font> | <font color='blue'>1</font><font color='red'>4</font><font color='blue'>4</font><font color='red'>1</font> | <font color='blue'>4</font><font color='red'>4</font><font color='blue'>1</font><font color='red'>1</font> | <font color='blue'>1</font><font color='red'>1</font><font color='blue'>4</font><font color='red'>4</font> |

- Even though we have `16` different permutations of `1144` that are balanced permutations. We only have `4` distinct balanced permutations, which are `4114`, `1441`, `4411`, `1144`

**Efficiently computing by using combinatorics and modular arithmetics**

- Since the computing of `dp[haftSum][k]` involes factorial. It would be wise that we pre-compute the factorial `i!`, for `i` from `1` to `n` and store it in array for later accessing.

- Furthermore, since the result must be `modulo 10`<sup>`9`</sup>` + 7`. We compute division of factorial $\frac{a}{b!}$ in modular arithmetics for better performance. Let `c = 10`<sup>`9`</sup>` + 7`

- Particularly, in modular arithmetics, we can not directly perform division `a/b! modulo c` as it may result float number. The division `(a/b!) mod c` is equivalent to multiplying `a` by the **modular inverse** of `b mod c`:

   $$ (a\,\div\,b)\,mod\,c = (a\,×\,b^{-1})\,mod\,c $$

- To compute modular inverse of number `b` modulo `c`, we use the Extended Euclidean Algorithm:

  $$b^{-1}\,(mod\,c) = c - (c\,//\,b) × (c\,mod\,b)^{-1}\,(mod\,c)$$

- To compute modular inverse of factorial `b!` modulo `c`, we compute as it base on `(b-1)!` and `b`<sup>`-1`</sup>

  $$(b!)^{-1}\,(mod\,c) = ((b−1)!)^{-1}\times b^{−1}\,(mod\,c)$$

- We can pre-compute modular inverse of number from `1` to `n` modulo `c` and store them first. Then we compute modular inverse factorial of `i!` from `2` to `n` based on previous value `(i-1)!` and pre-computed modular inverse of `i` modulo `c` above

### Analysis

- Time complexity:

  - Let `m = sum(n) // 2`

  - Time for pre-comuting factorial, modular inverse, duplicating removal: $O(n)$

  - Time for computing `dp` array: $O(n×m×n÷2) = O(n^2×m)$

  - Total time complexity: $O(n+n^2×m)$


### Implementation

In [9]:
from re import L
from typing import List, Optional
from collections import defaultdict
from collections import deque
from heapq import heapify, heappush, heappop
from queue import PriorityQueue
from collections import Counter
import math

class Solution:

    MOD = 10**9 + 7

    def countBalancedPermutations(self, num: str) -> int:
        sumDigits = sum(int(d) for d in num)
        if sumDigits % 2 != 0:
            return 0

        haftSum = sumDigits // 2
        k = len(num) // 2

        n = len(num)

        # pre-compute factorial
        factorial = [1] * (n + 1)
        for i in range(1, n + 1):
            factorial[i] = (factorial[i - 1] * i) % self.MOD

        # pre-compute modular inverse of b modulo c
        inverse = [1] * (n + 1)
        for i in range(2, n + 1):
            inverse[i] = self.MOD - self.MOD // i * inverse[self.MOD % i] % self.MOD

        # pre-compute modular inverse of factorial b! modulo c
        inverseFactorial = [1] * (n + 1)
        for i in range(1, n + 1):
            inverseFactorial[i] = inverseFactorial[i-1] * inverse[i] % self.MOD

        # computing using dp array
        dp = [[0] * (k + 1) for _ in range(haftSum + 1)]
        dp[0][0] = 1
        frequencies = [0] * 10
        for d in map(int, num):
            frequencies[d] += 1
            for i in range(haftSum, d - 1, -1):
                for j in range(k, 0, -1):
                    dp[i][j] = dp[i][j] + dp[i - d][j-1]

        result = dp[haftSum][k] * factorial[k] * factorial[n - k] % self.MOD

        # remove duplicating
        for freq in frequencies:
            result = result * inverseFactorial[freq] % self.MOD

        return result

In [10]:
run_test_cases(Solution(), "countBalancedPermutations")